In [9]:
# Nome: João Paulo Peixoto Castro
# Número: PG53929
# Curso: Mestrado em Engenharia Informática

In [4]:
from z3 import *

x={}
cores = ["azul","verde","cinzento","preto"]
for a in range(1,10):
    x[a]={}
    for c in cores:
        x[a][c] = Bool("%d,%s" % (a,c))

s = Solver()

# Cada azulejo tem uma ´unica cor = cada azulejo tem pelo menos e no maximo uma cor
#cada azulejo pelo menos uma cor
for a in range(1,10):
    s.add(Or([x[a][cor] for cor in cores]))

size = len(cores)
combs={}
for i in range(0,size):
    u = cores.pop(0)
    combs[u]=[]
    for j in range(0,len(cores)):
        combs[u].append(cores[j])
    cores.append(u)

#cada azulejo no maximo uma cor
for a in range(1,10):
    for c1 in combs.keys():
        s.add(Implies(x[a][c1],And([Not(x[a][c2]) for c2 in combs[c1]])))

# Os azulejos das extremidades tˆem que ter a mesma cor
s.add(Or([And(x[1][cor],x[9][cor]) for cor in cores]))

# Todas as cores tˆem que ser usadas.
s.add(And([Or([x[a][c] for a in range(1,10)]) for c in cores]))

# O azulejo do meio s´o pode ser pintado de cinzento ou preto.
s.add(Or(x[5]["cinzento"],x[5]["preto"]))

# O pen´ultimo azulejo da sequˆencia tem que ser preto, excepto se o ´ultimo estiver pintado de preto.
# not x[9][preto] -> x[8][preto]
s.add(Implies(Not(x[9]["preto"]),x[8]["preto"]))

# Azulejos seguidos n˜ao podem ser pintados da mesma cor.
for c in cores:
    for i in range(1,9):
        s.add(Implies(x[i][c],Not(x[i+1][c])))

# Caso o primeiro azulejo seja verde e o terceiro n˜ao seja preto, o segundo ter´a que ser cinzento ou preto.
s.add(Implies(And(x[1]["verde"],Not(x[3]["preto"])),Or(x[2]["cinzento"],x[2]["preto"])))

# Os azulejos na terceira e na s´etima posi¸c˜ao s´o podem ser pintados de azul ou de verde.
s.add(Or(x[3]["azul"],x[3]["verde"]))
s.add(Or(x[7]["azul"],x[3]["verde"]))


s.push()
# 1. Codifique o problema num solver e comprove que o conjunto de f´ormulas ´e consistente.
if s.check() == sat:
    m = s.model()
    print("O conjunto de fórmulas é consistente sendo que podemos compravá-lo pela seguinte atribuição de cores a cada um dos azulejos presente abaixo.\nUm possível conjunto de atribuições que obedecem às regras pré-definidas são as seguintes.:\n")
    for a in range(1,10):
        for c in cores:
            if is_true(m[x[a][c]]):
                print("O azulejo %d fica pintado de %s" % (a,c))
else:
  print("Não existe solução.")


O conjunto de fórmulas é consistente sendo que podemos compravá-lo pela seguinte atribuição de cores a cada um dos azulejos presente abaixo.
Um possível conjunto de atribuições que obedecem às regras pré-definidas são as seguintes.:

O azulejo 1 fica pintado de preto
O azulejo 2 fica pintado de azul
O azulejo 3 fica pintado de verde
O azulejo 4 fica pintado de azul
O azulejo 5 fica pintado de cinzento
O azulejo 6 fica pintado de azul
O azulejo 7 fica pintado de verde
O azulejo 8 fica pintado de azul
O azulejo 9 fica pintado de preto


In [5]:
# 2. Com a ajuda do solver, pronuncie-se quanto `a veracidade das seguintes afirma¸c˜oes:

# Para verificar a veracidade de cada uma das afirmações temos que proceder à negação da mesma e adicionar essa restrição ao conjunto de regras anteriores
# se o resultado for UNSAT quer dizer que é verdade

#(a) Se o primeiro azulejo for preto, o pen´ultimo azulejo tem que ser azul
print("---------------------------------------")
s.add(Not(Implies(x[1]["preto"],x[8]["azul"])))
s.push()
if s.check() == sat:
    print("A afirmação 2.(a) NÃO é verdadeira.")
else:
  print("A afirmação 2.(a) é verdadeira.")

s.pop()

#(b) Algum dos dois ´ultimos azulejos tem que ser preto.
s.add(Not(Or(x[8]["preto"],x[9]["preto"])))
s.push()
if s.check() == sat:
    print("A afirmação 2.(b) NÃO é verdadeira.")
else:
  print("A afirmação 2.(b) é verdadeira.")

s.pop()

# (c) Algum dos trˆes primeiros azulejos poder´a ser azul.
# nenhum dos tres primeiros azulejos poderá ser azul
s.add(And(Not(x[1]["azul"]),Not(x[2]["azul"]),Not(x[3]["azul"])))
s.push()

if(s.check()==sat):
    print("A afirmação 2.(c) NÃO é verdadeira.")
else:
    print("A afirmação 2.(c) é verdadeira.")
print("---------------------------------------")

s.pop()

---------------------------------------
A afirmação 2.(a) NÃO é verdadeira.
A afirmação 2.(b) é verdadeira.
A afirmação 2.(c) é verdadeira.
---------------------------------------


In [12]:
num=0
# 3 . Acrescente c´odigo ao programa de modo a saber todas as colora¸c˜oes que s˜ao poss´ıveis fazer aos azulejos. Quantas s˜ao?
s.pop()
while(s.check()==sat):
    m = s.model()
    num+=1
    aux=[]
    for v in m:
        aux.append(v() != m[v])
    s.add(Or([m for m in aux]))

print("Existem %d soluções possíveis" % num)
print("---------------------------------------")

Existem 360 soluções possíveis
---------------------------------------
